In [2]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score, cross_val_predict

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
diabetes = datasets.load_diabetes()

In [4]:
# View dict elements
diabetes.keys()

# View Data
print(diabetes.data.shape)
col_names = "age sex bmi map tc ldl hdl tch ltg glu".split()
data_frame = pd.DataFrame(diabetes.data, columns=col_names)
y = diabetes.target
sns.pairplot(data_frame)

(442, 10)


In [5]:
# Check for extreme/suspicious values
for col_name in col_names:
    print(col_name,":",min(data_frame[col_name]),np.percentile(data_frame[col_name], 
        [25, 50, 75]),max(data_frame[col_name]))

age : -0.107225631607358 [-0.03729927  0.00538306  0.03807591] 0.110726675453815
sex : -0.044641636506989 [-0.04464164 -0.04464164  0.05068012] 0.0506801187398187
bmi : -0.0902752958985185 [-0.03422907 -0.00728377  0.03124802] 0.17055522598066
map : -0.112399602060758 [-0.03665645 -0.00567061  0.03564384] 0.132044217194516
tc : -0.126780669916514 [-0.03424784 -0.00432087  0.02835801] 0.153913713156516
ldl : -0.115613065979398 [-0.0303584  -0.00381907  0.02984439] 0.198787989657293
hdl : -0.10230705051742 [-0.03511716 -0.00658447  0.0293115 ] 0.181179060397284
tch : -0.076394503750001 [-0.03949338 -0.00259226  0.03430886] 0.185234443260194
ltg : -0.126097385560409 [-0.03324879 -0.00194763  0.03243323] 0.133598980013008
glu : -0.137767225690012 [-0.03317903 -0.0010777   0.02791705] 0.135611830689079


In [6]:
regr = linear_model.LinearRegression()

# LOO Cross-Validation 
scores = cross_val_score(regr, data_frame, y, cv=5)
print(scores)

[0.42955643 0.52259828 0.4826784  0.42650827 0.55024923]


First try LOO Cross-Validation using all possible models with 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 variables. For each run, calculate $$RMSE = \sqrt{\frac{1}{n}\sum \limits_{i=1}^n (y_i - \widehat{y}_{(-i)})^2}$$

Then try a forward selection algorithm and see what it picks. Use the AIC criterion (or BIC).

If time try LASSO for variable selection

In [23]:
import itertools
from sklearn.model_selection import LeaveOneOut


def looRMSE(X, y):
    loo = LeaveOneOut()
    rmse = 0
    for train, test in loo.split(X):
        regr = linear_model.LinearRegression()
        regr.fit(X[train], y[train])
        rmse += (regr.predict(X[test])[0] - y[test][0])**2
        
    rmse = np.sqrt(rmse/(X.shape[1]))
    return rmse    
        
     
tested = []
result = []
for i in range(len(col_names)):
    for j in range(1, i + 1):
        testCombos = itertools.combinations(np.arange(i), j)
        for c in testCombos:
            if set(c) in tested:
                continue
            tested.append(set(c))
            testSet = data_frame.values[:, c]
            result.append(looRMSE(testSet, y))
            print("{}, {}".format(set(c), result[-1]))
        
    
    
        

{0}, 1596.774449474856
{1}, 1624.8189481590152
{0, 1}, 1131.5784732490818
{2}, 1316.7995055401027
{0, 2}, 928.5846750652731
{1, 2}, 933.2142502924809
{0, 1, 2}, 759.6196500086767
{3}, 1458.934843879105
{0, 3}, 1032.365988833729
{1, 3}, 1031.177074097785
{2, 3}, 895.6060651401256
{0, 1, 3}, 842.1762054511173
{0, 2, 3}, 732.6014447783236
{1, 2, 3}, 730.5869116565074
{0, 1, 2, 3}, 633.7176706496938
{4}, 1588.8837635570806
{0, 4}, 1114.6318742456078
{1, 4}, 1125.3407650540448
{2, 4}, 929.9379256624007
{3, 4}, 1026.3552079825456
{0, 1, 4}, 912.0523440327147
{0, 2, 4}, 758.3645186013586
{0, 3, 4}, 839.3406067863378
{1, 2, 4}, 760.9868097799134
{1, 3, 4}, 837.7988840884025
{2, 3, 4}, 732.2574768755429
{0, 1, 2, 4}, 658.0332514128692
{0, 1, 3, 4}, 726.5109147273132
{0, 2, 3, 4}, 635.418214541285
{1, 2, 3, 4}, 633.6159906002075
{0, 1, 2, 3, 4}, 567.7554107444533
{5}, 1601.164347528578
{0, 5}, 1120.6549147314113
{1, 5}, 1134.6263609309271
{2, 5}, 933.0476537513878
{3, 5}, 1028.2898207970418
{4, 

In [ ]:
# Forward selection with AIC criterion

def calculate_aic(y, yhat, k):
    e = np.sum((y-yhat)**2)
    aic = 2*k - 2*np.log(e)
    return aic
    

In [8]:
# View histogram of outcome variable
plt.hist(np.sqrt(y_train))

NameError: name 'y_train' is not defined

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(diabetes_X_train, diabetes_y_train)

In [ ]:
data_frame.shape[0]

In [ ]:
y